In [23]:
# pip install beautifulsoup4

In [17]:
from bs4 import BeautifulSoup
import uuid
import requests
import os

with open('_input.txt', 'r', encoding='utf') as f:
  data = f.read()

html = BeautifulSoup(data, 'lxml')
html = html.find('div', class_='flexlayout__tab')

problem_name = html.find('div').find('div').find('div').find('div').find('div').find("a").get_text()
problem_name = problem_name.split('.')[1:]
problem_name = ".".join(problem_name).strip()
problem = html.find('div', class_='elfjS')
images = problem.find_all('img')
img = []
if os.path.exists("assets"):
  files = os.listdir("assets")
  for file in files:
    os.remove(os.path.join("assets", file))
  os.removedirs("assets")
os.makedirs("assets", exist_ok=True)
for image in images:
  src = image.get('src')
  file_extension = src.split(".")[-1]
  file_name = uuid.uuid4().__str__().replace("-", "")
  response = requests.get(src)
  if response.status_code == 200:
    with open(f"assets/{file_name}.{file_extension}", "wb") as f:
      f.write(response.content)
  else:
    response = requests.get(src)
    if response.status_code == 200:
      with open(f"{file_name}.{file_extension}", "wb") as f:
        f.write(response.content)
  img.append({
    "source": src,
    "target": f"assets/{file_name}.{file_extension}"
  })

template = f"""
<h3>{problem_name}</h3>
{problem}
"""

template = template \
  .replace(" class=\"elfjS\" data-track-load=\"description_content\"", "") \
  .replace(" class=\"example\"", "")

for image in img:
  template = template.replace(image['source'], image['target'])

f = open("_question.md", "w", encoding='utf-8')
f.write(template)
f.close()
f = open("README.md", "w", encoding='utf-8')
f.write(template)
f.close()